In [ ]:
import os
import numpy as np
import pandas as pd
import pandavro as pdx

OUTPUT_PATH='{}/example.avro'.format('../files_avro')

df = pd.DataFrame({
    "Boolean": [True, False, True, False],
    "pdBoolean": pd.Series([True, None, True, False], dtype=pd.BooleanDtype()),
    "Float64": np.random.randn(4),
    "Int64": np.random.randint(0, 10, 4),
    "pdInt64":  pd.Series(list(np.random.randint(0, 10, 3)) + [None], dtype=pd.Int64Dtype()),
    "String": ['foo', 'bar', 'foo', 'bar'],
    "pdString": pd.Series(['foo', 'bar', 'foo', None], dtype=pd.StringDtype()),
    "DateTime64": [pd.Timestamp('20190101'), pd.Timestamp('20190102'),
                    pd.Timestamp('20190103'), pd.Timestamp('20190104')]
})

pdx.to_avro(OUTPUT_PATH, df)
saved = pdx.read_avro(OUTPUT_PATH)



In [69]:
import json
import pymysql
from sqlalchemy import create_engine

with open('..\security\parameters.json', 'r') as json_file:
    parameters = json.load(json_file)
    
db_user = parameters['DB_USER']
db_pass = parameters['DB_PASS']
db_name = parameters['DB_NAME']
db_host = parameters['DB_HOST']
db_port = parameters['DB_PORT']

def connect_database (db_user, db_pass, db_name, db_host):
    connection = pymysql.connect(
        host        = db_host,
        user        = db_user, 
        password    = db_pass,
        db          = db_name,
    )
    return connection

def connect_engine(db_user, db_pass, db_name, db_host, db_port):
    url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(db_user, db_pass, db_host, db_port, db_name)
    sqlEngine = create_engine(url)
    connection    = sqlEngine.connect()
    return connection

conn = connect_database (db_user, db_pass, db_name, db_host)
conn2 = connect_engine (db_user, db_pass, db_name, db_host, db_port)

In [ ]:
def return_tables(conection):
    cursor = conection.cursor()
    cursor.execute("SELECT table_name FROM information_schema.tables a WHERE TABLE_SCHEMA  = 'mydbpoc'")
    tablas = cursor.fetchall()
    cursor.close()
    ls = []
    for td in list(tablas): 
        ls.append(list(td)[0])
    return ls  

In [ ]:
tablas = return_tables(conn)
print(tablas)
table_name = 'jobs'
query = ("SELECT * FROM {}").format(table_name)
df_table = pd.read_sql(query, con = conn)
print (df_table)

pdx.to_avro(OUTPUT_PATH, df)


In [75]:
def backup_avro(conection):
    tables = return_tables(conection)
    for table in tables:
        df_table = pd.DataFrame()
        table_name = table
        query = ("SELECT * FROM {}").format(table_name)
        df_table = pd.read_sql(query, con = conn)
        pdx.to_avro('../files_avro/{}.avro'.format(table_name), df_table)

def rest_avro_table(engine, table_name):
    engine.execute('TRUNCATE TABLE {}'.format(table_name))
    data = pdx.read_avro('../files_avro/{}.avro'.format(table_name))
    data.to_sql(table_name, engine, if_exists='append',index=False)

rest_avro_table(conn2, 'mytable', )


None
